# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [195]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [196]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("./pset3_inputdata/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [197]:
## your code to subset to one press release and take the string
doj_sub = doj[doj["id"] == "17-1204"]
pharma = str(doj_sub.iloc[0]["contents"])
pharma

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [198]:
## your code here to restrict to alpha
# A
nopunc = ""
for c in pharma:
    if c.isalpha() or c == ' ':
        nopunc += c
#nopunc

# B
tokens = word_tokenize(nopunc) # Generate list of tokens
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens
#tokens_pos

# C
# take just the adjectives
all_adj = [one_tok[0] for one_tok in tokens_pos 
                if "JJ" in one_tok[1]]

# put adjectives in a data frame
df_adj = pd.DataFrame();
df_adj["adjectives"] = all_adj

# create new dataframe with unique adjectives and counts
def_adj_vcounts = pd.DataFrame(df_adj.adjectives.value_counts(), columns=["count"]);
def_adj_vcounts.reset_index().head()


,adjectives,count
0,former,8
1,opioid,5
2,nationwide,4
3,addictive,3
4,other,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [199]:
## your code here for part A
spacy_pharma = nlp(pharma)
for one_tok in spacy_pharma.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DAT

In [200]:
## your code here for part B
spacy_pharma_unique = [ent.text for ent in spacy_pharma.ents if ent.label_ == "LAW"]
ser_unique = pd.Series(spacy_pharma_unique).unique()
for one_tok in ser_unique:
    print("Entity: " + one_tok + "; NER tag: LAW")

Entity: RICO; NER tag: LAW
Entity: the Controlled Substances Act; NER tag: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

### RICO
RICO (Racketeer Influenced and Corrupt Organizations Act) is a law that allows the government to prosecute all those involved with larger crime enterprises, even those that aren't directly committing the crimes.  RICO could apply to a pharmaceutical kickback case because it outlaws behind-the-scenes agreements and manipulative cooperation to benefit the pharmaceutical companies and their partners.  Even if all of the people involved aren't individually doing something illegal, they are collectively working together to do something illegal.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [201]:
## your code here
spacy_pharma_dates = [ent.text for ent in spacy_pharma.ents if ent.label_ == "DATE"]
year_regex = r".*years|.*year"

years = [re.findall(year_regex, ent) 
               for ent in spacy_pharma_dates]
years = [year for year in years if year]

years

[['last year'], ['20 years'], ['three years'], ['five years'], ['three years']]

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [202]:
## your code here
sentence_regex = r".{0,70}years.{0,70}|.{0,30}year.{0,30}"
sentence = [re.findall(sentence_regex, ent.text) 
               for ent in spacy_pharma.sents]
sentence = [sent for sent in sentence if sent]

sentence

[['nthetic opioid overdoses last year, and millions are addicted to'],
 ['mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, '],
 [' the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']]

### Sentence
The CEO may face a 20-year sentence and 3 years of supervised release if convicted.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [203]:
## your code here for subsetting
doj_subset = doj[(doj["topics_clean"] == "Civil Rights") | (doj["topics_clean"] == "Hate Crimes") | (doj["topics_clean"] == "Project Safe Childhood")]
doj_subset

,id,title,contents,date,topics_clean,components_clean
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project Safe Childh

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [204]:
## your code here to define function
def analyze(press_str):
    press_str_sp = nlp(press_str)
    ents = [ent.text for ent in press_str_sp.ents]
    regex = "|".join(re.escape(ent) for ent in ents)
    no_ents = re.sub(regex, "", press_str)
    ## initialize a scorer
    sent_obj = SentimentIntensityAnalyzer()
    ## score one listing
    sentiment_analysis = sent_obj.polarity_scores(no_ents)
    return sentiment_analysis

In [ ]:
## your code here executing the function
pol_scores = [analyze(str(text)) for text in doj_subset.contents]
pol_scores

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [233]:
## your code here
doj_subset_wscore = doj_subset
doj_subset_wscore["neg"] = [row['neg'] for row in pol_scores]
doj_subset_wscore["neu"] = [row['neu'] for row in pol_scores]
doj_subset_wscore["pos"] = [row['pos'] for row in pol_scores]
doj_subset_wscore["compound"] = [row['compound'] for row in pol_scores]

# sort and display
doj_subset_wscore = doj_subset_wscore.sort_values("neg", ascending=False)
doj_subset_wscore[["id", "contents", "neg"]]

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.334
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.301
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and 

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [234]:
## agg and find the mean compound score by topic
# D
doj_subset_wscore.groupby("topics_clean").agg({"compound": "mean"})

,compound
topics_clean,
Civil Rights,-0.094465
Hate Crimes,-0.935971
Project Safe Childhood,-0.660416


# Analysis
All of the topics are pretty negative, which makes sense because they are about prosecutions.  The variation (ei. less negative project safe childhood and much less negative civil rights) could be due to the the fact that the two less negative topics include outlook to a more positive future, while hate crimes focuses on the damage done.  

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [ ]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [ ]:
## your code defining a text processing function
# import
porter = PorterStemmer()

# create stopwords list
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords

# preprocess the string - function
def preprocess(content):
    content_lower = content.lower()
    nostop_listing = [word for word in wordpunct_tokenize(content_lower) if word not in list_stopwords_new]
    stem_listing = [porter.stem(token) 
                            for token in nostop_listing 
                            if token.isalpha() and 
                            len(token) >= 4]
    return " ".join(stem_listing)
    

In [ ]:
## your code executing the function
doj_subset_wscore["processed_text"] = [preprocess(str(x)) for x in doj_subset_wscore["contents"]]

In [ ]:
## your code showing the examples
doj_subset_wscore[(doj_subset_wscore["id"] == "16-718") | (doj_subset_wscore["id"] == "16-217")][["id", "contents", "processed_text"]]

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [ ]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [221]:
# your code here
# A
dt_matrix = create_dtm(list(doj_subset_wscore["processed_text"]), doj_subset_wscore[["id", "compound", "topics_clean"]])
dt_matrix

def get_topwords(df):
    dt_matrix_sub = create_dtm(list(df["processed_text"]), df[["id", "compound", "topics_clean"]])
    sums = dt_matrix_sub.iloc[:,4:].sum().sort_values(ascending=False)
    return sums

# B
cutoff_high = doj_subset_wscore["compound"].quantile(0.95)
top_5 = doj_subset_wscore[doj_subset_wscore["compound"] > cutoff_high]
get_topwords(top_5).head(10)

# C
cutoff_low = doj_subset_wscore["compound"].quantile(0.05)
bot_5 = doj_subset_wscore[doj_subset_wscore["compound"] < cutoff_low]
get_topwords(bot_5).head(10)

# D
for topic in doj_subset_wscore["topics_clean"].unique():
    print(topic + ": ")
    spec_topic = doj_subset_wscore[doj_subset_wscore["topics_clean"] == topic]
    get_topwords(spec_topic).head(10)
    




,index,id,compound,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealou,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,329,14-248,-0.9951,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,572,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11248,10-1194,-0.9990,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,10286,15-1559,0.8481,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,1857,17-271,0.9909,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,6787,17-132,0.9794,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,11085,15-667,0.7003,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


agreement     175
enforc        121
state         118
commun        112
ensur         110
disabl        107
court          91
servic         90
settlement     87
student        87
dtype: int64

assault     181
crime       172
victim      164
offic       155
hate        129
defend      123
sentenc     104
conspir     104
american     95
feder        95
dtype: int64

Hate Crimes: 


victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

Civil Rights: 


offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
gener        414
dtype: int64

Project Safe Childhood: 


child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [223]:
# your code here
# A
text_tokens  = [wordpunct_tokenize(text) for text in doj_subset_wscore["processed_text"]]
text_dict = corpora.Dictionary(text_tokens)
text_fromdict = [text_dict.doc2bow(text) for text in text_tokens]
ldamod = gensim.models.ldamodel.LdaModel(text_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         random_state = 0,
                                         per_word_topics = True)
topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)
for tnum, topic in ldamod.show_topics(num_words=15, formatted=False):
    print(f"Topic {tnum}:")
    print(", ".join([word for word, prob in topic]))
    
    

(0, '0.011*"victim" + 0.010*"charg" + 0.010*"crime" + 0.009*"defend" + 0.009*"sentenc" + 0.009*"prosecut" + 0.009*"hate" + 0.008*"guilti" + 0.008*"feder" + 0.008*"indict" + 0.008*"said" + 0.007*"year" + 0.007*"american" + 0.007*"assault" + 0.006*"prison"')
(1, '0.019*"child" + 0.013*"exploit" + 0.011*"offic" + 0.011*"sexual" + 0.009*"crimin" + 0.009*"prosecut" + 0.009*"safe" + 0.009*"project" + 0.009*"childhood" + 0.009*"sentenc" + 0.008*"pornographi" + 0.008*"victim" + 0.008*"children" + 0.007*"feder" + 0.007*"investig"')
(2, '0.011*"hous" + 0.011*"discrimin" + 0.010*"disabl" + 0.007*"enforc" + 0.007*"agreement" + 0.007*"said" + 0.007*"state" + 0.006*"court" + 0.006*"alleg" + 0.006*"settlement" + 0.005*"requir" + 0.005*"feder" + 0.005*"violat" + 0.005*"fair" + 0.005*"gener"')
Topic 0:
victim, charg, crime, defend, sentenc, prosecut, hate, guilti, feder, indict, said, year, american, assault, prison
Topic 1:
child, exploit, offic, sexual, crimin, prosecut, safe, project, childhood, sen

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [217]:
## your code here to get doc-level topic probabilities 
# A
topic_probs = [ldamod.get_document_topics(text, minimum_probability=0) for text in text_fromdict]
len(topic_probs)
topic_probs

717

[[(0, 0.99854606), (1, 0.0007789826), (2, 0.0006749342)],
 [(0, 0.9992845), (1, 0.00038333551), (2, 0.00033213358)],
 [(0, 0.49031565), (1, 0.5092508), (2, 0.000433527)],
 [(0, 0.99932337), (1, 0.00036256108), (2, 0.00031413423)],
 [(0, 0.81331927), (1, 0.00024690013), (2, 0.18643388)],
 [(0, 0.9991188), (1, 0.00047216393), (2, 0.00040909735)],
 [(0, 0.9985949), (1, 0.0007528236), (2, 0.0006522694)],
 [(0, 0.8863447), (1, 0.11334747), (2, 0.00030787164)],
 [(0, 0.9987853), (1, 0.00065080123), (2, 0.0005638738)],
 [(0, 0.0023204386), (1, 0.002023406), (2, 0.9956562)],
 [(0, 0.6577982), (1, 0.00042991916), (2, 0.3417719)],
 [(0, 0.999562), (1, 0.00023466723), (2, 0.00020332288)],
 [(0, 0.9994039), (1, 0.00031940444), (2, 0.00027674183)],
 [(0, 0.9990446), (1, 0.0005119312), (2, 0.00044355303)],
 [(0, 0.80445516), (1, 0.00031662267), (2, 0.1952282)],
 [(0, 0.9994579), (1, 0.000290453), (2, 0.00025165745)],
 [(0, 0.99890256), (1, 0.0005879801), (2, 0.0005094441)],
 [(0, 0.99945086), (1, 0.

In [237]:
## your code here to add those topic probabilities to the dataframe
# B
def largest(row):
    if row[0][1] > row[1][1] and row[0][1] > row[2][1]:
        return 1
    elif row[1][1] > row[0][1] and row[1][1] > row[2][1]:
        return 2
    else: #row[2][1] > row[1][1] and row[2][1] > row[0][1]:
        return 3

doj_subset_wscore["top_topic"] = [largest(row) for row in topic_probs]
doj_subset_wscore["top_topic"]


329      1
572      1
11593    2
501      1
11248    1
        ..
10286    3
1857     3
6787     3
11085    3
6981     3
Name: top_topic, Length: 717, dtype: int64

In [241]:
## your code here to summarize the topic proportions for each of the topics_clean 
# C
hate_tot = 0
hate_acc = 0
child_tot = 0
child_acc = 0
civil_tot = 0
civil_acc = 0
for idx, row in doj_subset_wscore.iterrows():
    if row["topics_clean"] == "Hate Crimes":
        hate_tot += 1
        if row["top_topic"] == 1:
            hate_acc += 1
    elif row["topics_clean"] == "Project Safe Childhood":
        child_tot += 1
        if row["top_topic"] == 2:
            child_acc += 1
    elif row["topics_clean"] == "Civil Rights":
        civil_tot += 1
        if row["top_topic"] == 3:
            civil_acc += 1

print(f"Hate Crime: {hate_acc/hate_tot}")
print(f"Project Safe Childhood: {child_acc/hate_tot}")
print(f"Civil Rights: {civil_acc/civil_tot}")
doj_subset_wscore

Hate Crime: 0.9959349593495935
Project Safe Childhood: 0.6707317073170732
Civil Rights: 0.6688524590163935


,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,top_topic
329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.334,0.633,0.033,-0.9951,1
572,13-312,Aryan Brother Inmate Sentenced for Federal Hate Crime for Assaulting Fellow Inmate,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",2013-03-14T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civ

#### Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)
Some topics, such as Hate Crime, map more cleanly to an estimated topic than others, such as Project Safe Childhood and Civil Rights.  One reason why this could be is because the Hate Crimes press releases were much more negative than the others, which could result in more accurate grouping because it is more distinguishable to the other groups, which are less negative.  For example, in the case with id 8207, in the Protect Safe Childhood topic, there are many words such as child, innocence, and protect that have a positive connotation and are mixed into the description.  On the other hand, case id 501, which is a hate crime, is full of words such as victim, injuries, life-threatening and very negative diction, without much of the positive diciton found in other topics.

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## your code here 

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# your code here

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 